# Zeitevolution von Topics

In den letzten Kapiteln hast du viel zu unterschiedlichen Topic Models gelernt. Allerdings waren diese alle statisch. Dabei sind die Newsticker-Meldungen *zeitabhängig*.

Es stellt sich als schwierig heraus, *zeitabhängige Topic Models* zu berechnen, weil sich die einzelnen Topics nicht mehr zuordnen lassen. Stattdessen ist es geschickter, wenn du ein einziges Topic Model berechnest und überprüfst, wie sich die Anteile an den Topics im Laufe der Zeit verschieben.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Anschließend führst du die Vektorisierung durch. Das kann auch wieder einen Augenblick dauern:

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=5)
tfidf_vectors = tfidf_vectorizer.fit_transform(df["nav"])
tfidf_vectors

Durch die Nutzung von `min_df=5` ist die Matrix einigermaßen übersichtlich geblieben!

## Topic Model mit NMF berechnen

Die Aufrufsyntax des Topic Models ist sehr ähnlich zu der des `TfidfVectorizers`, nur dass du hier keine Transformation durchführen musst, daher heißt die Methode nur `fit`. Der Aufruf kann ein paar Sekunden dauern:

In [ ]:
from sklearn.decomposition import NMF

num_topics = 10

nmf = NMF(n_components = num_topics)
nmf.fit(tfidf_vectors)

Um die Topics übersichtlich darstellen zu können, kannst du eine Funktion nutzen, die einen separaten Dataframe dafür aufbaut. Dazu iterierst du über alle Topics (`n_components` im Topic Model) und ermittelst jeweils immer die wichtigsten Wörter. `argsort` sortiert aufsteigend, daher sind es die letzten Indizes im Array:

In [ ]:
def topics_table(model, feature_names, n_top_words = 20):
    word_dict = {}
    
    for i in range(model.n_components):
        # ermittle für jedes Topic die größten Werte
        words_ids = model.components_[i].argsort()[:-n_top_words-1:-1]
        words = [feature_names[key] for key in words_ids]
        # und füge die entsprechenden Worte im Klartext dem Dictionary hinzu
        word_dict['Topic #%02d' % i] = words;
    
    return pd.DataFrame(word_dict)

Schau dir jetzt die Tabelle mit den Themen an:

In [ ]:
topics_table(nmf, tfidf_vectorizer.get_feature_names())

Nun setzt du ein Feld im `DataFrame` auf den Monat:

In [ ]:
df["month"] = pd.to_datetime(df["datePublished"], utc=True).dt.strftime("%Y-%m")

Anschließend iterierst du über alle Monate und wendest das Topic Model für die Daten des entsprechenden Monats an. Die Anteil der Topics speicherst du in `month_data`:

In [ ]:
from tqdm.auto import tqdm
import numpy as np
month_data = []
for month in tqdm(np.unique(np.unique(df["month"]))):
    W_month = nmf.transform(tfidf_vectors[np.array(df["month"] == month)])
    month_data.append([month] + list(W_month.sum(axis=0)/W_month.sum()*100.0))

Damit die Topics nicht nur Nummern haben, merkst du dir die beiden wichtigsten Wörter für jedes Topic:

In [ ]:
topic_names = []
voc = tfidf_vectorizer.get_feature_names()
for topic in nmf.components_:
    important = topic.argsort()
    top_word = voc[important[-1]] + " " + voc[important[-2]]
    topic_names.append("Topic " + top_word)

Das fügst du nun alles in einem `DataFrame` zusammen und plottest das Ergebnis:

In [ ]:
df_month = pd.DataFrame(month_data, columns=["month"] + topic_names).set_index("month")
df_month.plot.area(figsize=(16,9))

Das Ergebnis ist nicht nur optisch schön, sonst auch gut interpretierbar. Am Anfang des Jahres scheint Apple und iPhone keine große Rolle gespielt zu haben, das hat sich später deutlich verändert zu Lasten des generischen Topics. Corona und die Warnapp haben auch ab März zugenommen, was auch der eigenen Wahrnehmung entspricht.

Neben diesen Erkenntnissen ist es besonders spannend, dass das alles *unüberwacht* und damit auch *unvoreingenommen* entstanden ist. Dieselbe Methode kannst du auch für andere (zeitabhängige) Daten einsetzen.

## Zeitevolution ist spannend

Die hier erklärten Methoden kannst du nicht nur für Topics, sondern auch für Cluster einsetzen. Dann bist du schon fast im Bereich der *Klassifikation*, mit dem wir uns im nächsten Kapitel beschäftigen.